In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import glob
import pandas as pd
from common_functions import get_parties, get_election_results, process_election

In [4]:
# These files come from http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html.
# Scroll down to "Extracción de datos" to obtain the zip files with the info at the
# ballot box level.
datafiles = glob.glob("/home/chema/Dropbox/data/elecciones/*.zip")
print(datafiles)

['/home/chema/Dropbox/data/elecciones/04201105_MESA.zip', '/home/chema/Dropbox/data/elecciones/04201505_MESA.zip', '/home/chema/Dropbox/data/elecciones/02201512_MESA.zip', '/home/chema/Dropbox/data/elecciones/02201606_MESA.zip']


De acuerdo a [este archivo](http://www.ine.es/daco/daco42/codmun/codmun11/11codmunmapa.htm), Madrid es la provincia 28, municipio 079. Tenemos que filtrar usando estos códigos.

In [5]:
# Find the doomed ballot boxes in the 2011 elections
el2011 = process_election(datafiles[0])

In [6]:
# Need to get the percentage of vote for a given table + section
el2011_total = el2011.groupby(['section_code', 'table_code', 'dist_code']).agg({'votes': 'sum'}).reset_index()
el2011_total['total_votes'] = el2011_total['votes']
del el2011_total['votes']
el2011 = pd.merge(el2011, el2011_total)
el2011['pct_votes'] = el2011['votes'] / el2011['total_votes']

In [7]:
# Now find the boxes that had the most vote to Falange
el2011.loc[el2011['party_name'] == "LA FALANGE", :].sort_values('pct_votes', ascending = False).head(5)

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes
27283,08,122620,LA FALANGE,28,012,U,079,349,702,0.497151
43033,11,122620,LA FALANGE,28,007,B,079,184,396,0.464646
57333,13,122620,LA FALANGE,28,203,A,079,84,404,0.207921
53683,13,122620,LA FALANGE,28,061,U,079,5,301,0.016611
1733,01,122620,LA FALANGE,28,062,U,079,8,487,0.016427


In [16]:
dist_code = '13'
section_code = '203'
table_code = 'A'
summary = el2011.loc[(el2011['section_code'] == section_code) \
                      & (el2011['dist_code'] == dist_code) \
                      & (el2011['table_code'] == table_code), :]\
    .sort_values('pct_votes', ascending = False)
display(summary)
summary.to_csv(f'/tmp/resultados_distrito_{dist_code}_seccion_{section_code}_mesa_{table_code}.csv')
# 1.0
# print(el2011.loc[(el2011['section_code'] == '012') & (el2011['dist_code'] == '08'), :].pct_votes.sum())

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes
57344,13,123438,POR UN MUNDO MAS JUSTO,28,203,A,079,150,404,0.371287
57347,13,123771,UNION POR LEGANES,28,203,A,079,131,404,0.324257
57333,13,122620,LA FALANGE,28,203,A,079,84,404,0.207921
57334,13,122876,PARTIDO ANTITAURINO CONTRA EL MALTRATO ANIMAL,28,203,A,079,27,404,0.066832
57327,13,121193,CIUDADANOS-PARTIDO DE LA CIUDADANIA,28,203,A,079,3,404,0.007426
57332,13,122515,IZQUIERDA UNIDA-LOS VERDES,28,203,A,079,2,404,0.004950
57325,13,120694,ALTERNATIVA ESPAÑOLA,28,203,A,079,1,404,0.002475
57326,13,121182,CIUDADANOS EN BLANCO,28,203,A,079,1,404,0.002475
57346,13,123652,UNIFICACION COMUNISTA DE ESPAÑA,28,203,A,079,1,404,0.002475
57343,13,123186,PARTIDO SOCIALISTA OBRERO ESPAÑOL,28,203,A,079,1,404,0.002475


In [18]:
# Sanity check: do we get the same global numbers as the webpage reports
# http://www.infoelectoral.mir.es/infoelectoral/min/busquedaAvanzadaAction.html
el2011.groupby('party_name').agg({'votes': 'sum'}).sort_values('votes', ascending = False)
# Yes, we do.

,votes
party_name,
PARTIDO POPULAR,756952
PARTIDO SOCIALISTA OBRERO ESPAÑOL,364600
IZQUIERDA UNIDA-LOS VERDES,163706
UNION PROGRESO Y DEMOCRACIA,119601
ECOLO VERDES,13425
CIUDADANOS EN BLANCO,10795
PARTIDO ANTITAURINO CONTRA EL MALTRATO ANIMAL,7071
POR UN MUNDO MAS JUSTO,6456
ALTERNATIVA ESPAÑOLA,4764
